In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM, Bidirectional, Dropout
from keras.metrics import AUC
from keras.models import Model
from keras import layers
from keras import Input

## 数据集：

data.csv 原始文件,记录细胞轨迹数据 \
id_link.csv 原始文件，记录细胞重叠事件 \

In [ ]:
# 导入原始数据并保存为csv格式
data = pd.read_csv('data/xyarr2D191211_001.txt',sep = r' +', header=None, engine='python')
data.columns = ['size','x','y','t','id','ovality']
data = data[['id','t','x','y','size','ovality']]

In [ ]:
# 查看轨迹数据的基本信息

def tra_describe(data):
    
    # data格式: clos :  id t x y size ovality   index无意义
    
    # num_id_frame :  每个 time_frame 中的 id 数
    # id_steps: id对应的步长
    # steps_dis: index是步长  ‘count’是该步长的id数 ‘cum_sum’是步长从大到小累计百分比
    # walk_range : 细胞的移动范围，有x方向、y方向
    

    # 查看每个 time_frame 中的 id 数
    num_id_frame = data[['t','id']].groupby(by = 't').count()
    num_id_frame.columns= ['num_id_frame']    

    #查看步长分布
    id_steps = data['id'].value_counts().to_frame() #id_count:  index是id 列为id对应的步数'steps'
    id_steps.columns= ['steps']

    steps_count = id_steps['steps'].value_counts().to_frame().sort_index(ascending=False) #index是步长，‘count’是步长出现的次数 
    steps_count.columns= ['count']
    steps_count['cum_sum'] = steps_count.cumsum()/steps_count.sum() #注意这里步长是倒序，方便计算cum_sum
    steps_dis = steps_count.sort_index()
    
    
    # 查看x的范围和y的范围分布  # 应该算路径而不是位移？

    walk_range = data.groupby(by = 'id').apply(max)[['x','y']] - data.groupby(by = 'id').apply(min)[['x','y']]
    walk_range['max_x_y'] = walk_range.max(axis = 1)
    walk_range['x_y'] = np.sqrt(walk_range['x']**2 + walk_range['y']**2)
    
    return num_id_frame, id_steps, steps_dis, walk_range


# clean data

# ！！！！需要有 data,id_steps,walk_range 三个数据表和 min_steps ,min_walk_range 两个参数 ！！！！

def get_data_clean(data,id_steps,walk_range,min_steps = 60 ,min_walk_range = 20) : 

    # min_steps 去除步长过小的细胞
    # min_walk_range 去除运动范围很小的细胞
    
    #满足条件的id
    
    id_total = list(data['id'].unique())
    
    id_filter = id_steps[id_steps['steps'] >= min_steps].join(
            walk_range[walk_range['max_x_y'] >= min_walk_range],  # 'max_x_y'可以换成“x_y”
            how = 'inner')
    
    data_clean = data.set_index('id').loc[id_filter.index].reset_index()
    
    print("设置的最小步长为: ", min_steps, ";  最小运动范围为: ", min_walk_range)
    print("被选出的id数量: ", len(id_filter))
    print("占总id数量的比例: ", len(id_filter)/len(id_total))
    print('已得到data_clean')
    
    return data_clean

In [ ]:
num_id_frame, id_steps, steps_dis, walk_range = tra_describe(data)

In [ ]:
# 查看移动范围分布
plt.figure(figsize=(8,5))
sns.distplot(walk_range['max_x_y'],bins = 100)
plt.title('distribution of walk_range')
plt.xlabel('walk_range')
plt.ylabel('proportion')

In [ ]:
# 查看步长分布
plt.figure(figsize=(8,5))
sns.distplot(id_steps,bins = 50)
plt.title('distribution of steps')
plt.xlabel('steps')
plt.ylabel('proportion')

In [ ]:
# 根据步长和移动范围分布确定截断点，获得clean之后的data
data_clean = get_data_clean(data,id_steps,walk_range,min_steps = 60 ,min_walk_range = 20)

In [ ]:
# 导入id_link数据
id_link = pd.read_csv('data/id_link191211_001.csv', header=None)
id_link.columns = ['id1','id2','id3','id4','type','id5','id6','id7','id8','t']

In [ ]:
# 重构 id_link 的格式，将一次完整的重叠又分裂的行为放在一行，去除只有重合/只有分裂的事件

#重叠
type0 = id_link[id_link['type'] == 0][['id1','id2','id3','id4','id5','t']].rename(columns={'id5' : 'id_overlap',
                                                                                           't':'t1'}) 
#分开           
type1 = id_link[id_link['type'] == 1][['id1','id5','id6','id7','id8','t']].rename(columns={'id1' : 'id_overlap',
                                                                                           't':'t2'}) 
#拼合
id_link_new = type0.join(type1.set_index('id_overlap'), on='id_overlap', how='inner') # how = 'outer'即可保留所有事件
id_link_new['overlap_time'] = id_link_new['t1']-id_link_new['t2']

In [ ]:
# 查看重叠时间长度的分布
plt.figure(figsize=(8,5))
sns.distplot(id_link_new['overlap_time'],bins =100)
plt.title('distribution of overlap_time')
plt.xlabel('overlap_time')
plt.ylabel('proportion')

In [ ]:
# 根据data clean的结果过滤一些未被选择的id
id_list = list(data_clean['id'].unique())
a = id_link_new[id_link_new['id1'].isin(id_list)]
b = a[a['id2'].isin(id_list)]
c = b[b['id5'].isin(id_list)]
id_link_filter = c[c['id6'].isin(id_list)].reset_index(drop = True)

print('原数据集发生完整的重合又分开事件的个数: ',len(id_link_new))
print('这些事件中前后四个id都在data_clean中的个数: ', len(id_link_filter))

# 数据集 总结：

data  原始文件 \
data_clean  去掉轨迹过短和移动范围过少的细胞  

id_link  原始文件 \
id_link_new  重构，完整的重叠又分开 \
id_link_filter 筛选过的id

# 单输入model 


#### 1)选取一个id连续轨迹的后30步作为碰撞前的轨迹 tra_pre
#### 2)选取一个id连续轨迹的前30步作为碰撞后的轨迹 tra_post

#### 3)将tra_pre，tra_post连接起来:

   #### 若这两段轨迹来自同一cell，则输出label=1；
   #### 若这两段轨迹来自不同cell，则输出label=0；


In [ ]:
# 获取轨迹数据，并去掉overlap期间的轨迹记录（物理运动规律可能会变化？？）
id_overlap_list = id_link_new['id_overlap'].to_list()
df = data_clean[ ~ data_clean.id.isin(id_overlap_list)]
id_list = df['id'].unique().tolist() #id列表
id_t0 = df.sort_values(['id','t']).groupby('id').head(1) #每个时间桢新出现的id
ts = list(id_t0['t'].unique()) #时间帧

In [ ]:
# 取出前30和后30步
# 注意!!! 使用head 和tail 时要保证数据按照时间排序 !!!!!!
steps = 30

df_post1 = df.sort_values(['id','t']).groupby('id').head(steps).set_index('id')  # 碰撞之后
df_pre1 = df.sort_values(['id','t']).groupby('id').tail(steps).set_index('id') # 碰撞之前

# 将前30的第一步的坐标移动到（0，0） 初始时间为0
# 将后30的最后一步移动到 （0，0） 结束时间为0
df_post2 = df_post1.sub(df_post1.groupby(by = 'id').head(1))  # sub 会对准index
df_pre2 = df_pre1.sub(df_pre1.groupby(by = 'id').tail(1))

### 构造label为1的数据集

In [ ]:
df_post3 = pd.concat([df_post2[['t','x','y']],df_post1[['size','ovality']]],axis = 1)
df_pre3 = pd.concat([df_pre2[['t','x','y']],df_pre1[['size','ovality']]],axis = 1)

# label = 1  将同一个id 的后30和前30拼接到一起 （注意：前后颠倒）
# 并去除 post中重复的（0，0）坐标
df_label1 = pd.concat([df_pre3,df_post3.groupby(by = 'id').tail(steps - 1)])
df1 = df_label1.reset_index().set_index(['id','t']).sort_index()   #注意要sort_index

### 构造label为0的数据集

In [ ]:
li = []

# 选择一些时间frame中的id 确保这些id一定代表不同的细胞

for t in ts:    # 如果不用t0会有很多重复的ID，因为同一个ID在不同的 t frame 中出现！！！！！
    
    id_list_t = id_t0[id_t0['t'] == t]['id'].to_list()  #t frame 中的那些细胞
    id_list_t.sort()
    
    i = 0
    while i < len(id_list_t)-1 :
        
        id1 = id_list_t[i]
        id2 = id_list_t[i+1]     #也和现实情况很像，因为一般发生overlap的细胞编号一般是连着的
        # id3 = id_list_t[i+2]

        tra = pd.concat([df_pre2.loc[id1],df_post2.loc[id2].tail(steps - 1)])
        others = pd.concat([df_pre1.loc[id1],df_post1.loc[id2].tail(steps - 1)])
        
        li.append(pd.concat([tra[['t','x','y']],others[['size','ovality']]],axis = 1))
        i = i+1  # 或者可以i+2利用更多id

df0_concat = pd.concat(li,keys = range(len(li)),names=['sample'])
df0 = df0_concat.reset_index().set_index (['sample','t'])[['x','y','size','ovality']].sort_index()

### 定义函数： input 张量化并划分训练集

In [ ]:
def Tensorize(df) :  # 输入需要张量化的dataframe, df.index=['id','t'] df.values = ['features']
    # 样本个数
    num_sample = df.index.unique(0).shape[0]
    # 时间步长
    timesteps = df.index.unique(1).shape[0]
    # 特征数量
    num_features = df.shape[1]

    #张量化
    ar = df.to_numpy().reshape((num_sample, timesteps, num_features)) #(sample, timesteps, features)
    
    print('shape of array is : ', ar.shape)
    
    return ar

def get_dataset_for_model(df0,df1):
    
    ar0 = Tensorize(df0)
    # label each sample with 0
    label0 = np.zeros(ar0.shape[0])

    ar1 = Tensorize(df1)
    # label each sample with 1
    label1 = np.ones(ar1.shape[0])

    ar = np.concatenate((ar1, ar0), axis = 0)
    label = np.concatenate((label1, label0), axis = 0)
    
    return ar,label

### 定义函数： 训练模型  （以3层Bi-LSTM为例）

In [ ]:
def train_model_single(input_train,label_train,epochs=4):
    
    #build model
    model = Sequential()
    model.add(
        Bidirectional(LSTM(64, return_sequences=True), input_shape=(None, input_train.shape[-1]))
    )
    model.add(
        Bidirectional(LSTM(64, return_sequences=True))
    )
    model.add(Bidirectional(LSTM(32)))

    model.add(Dense(32, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))

    #model.summary()
    
    
    # train model
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    history= model.fit(input_train, label_train, epochs=epochs, batch_size=64, validation_split=0.2)
    
    return model,history


def get_training_result(history):
# 绘制结果

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
    
def get_test_result(model,input_test,label_test):
    
    # evaluate on test dataset
    print("Evaluate on test data")
    results = model.evaluate(input_test, label_test, batch_size=128)
    print("test loss, test acc:", results)


    # 比较预测和真实的label
    test_result = pd.concat([pd.DataFrame(model.predict(input_test)),pd.DataFrame(label_test)],axis = 1)
    test_result.columns = ['predict','label']
    test_result = test_result.sort_values('predict').reset_index(drop = True)

    # 画图
    #plt.figure(figsize=(15, 10))
    #plt.scatter(test_result.reset_index()['index'],test_result['label'],s=0.2,c='red')
    #plt.scatter(test_result.reset_index()['index'],test_result['predict'],s=0.3,c='blue')    


## 选择特征输入，以(x,y,size)为例

In [ ]:
ar,label = get_dataset_for_model(df0[['x','y','size']],df1[['x','y','size']])
input_train, input_test, label_train, label_test = train_test_split(ar,label,test_size=0.2, random_state=420)
model4,history4 = train_model_single(input_train4,label_train4,epochs = 5)
get_training_result(history)
get_test_result(model4,input_test4,label_test4)

# 双输入模型

## 数据处理

In [ ]:
# 取出前30和后30步
# 注意!!! 使用head 和tail 时要保证数据按照时间排序

steps = 30

df_post1 = df.sort_values(['id','t']).groupby('id').head(steps).set_index('id')  # 碰撞之后
df_pre1 = df.sort_values(['id','t']).groupby('id').tail(steps).set_index('id') # 碰撞之前

# 将新的步长为30的轨迹起点移动到（0，0） 初始时间为0  
# 和单输入模型有区别
df_post2 = df_post1.sub(df_post1.groupby(by = 'id').head(1))  
df_pre2 = df_pre1.sub(df_pre1.groupby(by = 'id').head(1))

df_post3 = pd.concat([df_post2[['t','x','y']],df_post1[['size','ovality']]],axis = 1)
df_pre3 = pd.concat([df_pre2[['t','x','y']],df_pre1[['size','ovality']]],axis = 1)

In [ ]:
# 准备label = 0 数据集
input1_label0 = df_pre1.head(0)
input2_label0 = df_post1.head(0)

for t in ts:
    id_list_t = id_t0[id_t0['t'] == t]['id'].to_list()  #t frame 中的那些细胞
    id_list_t.sort()
    num_id = len (id_list_t)
    
    #错位一下
    df_pre_0 = df_pre3.loc[id_list_t[:num_id-1]] 
    df_post_0 = df_post3.loc[id_list_t[1:]]
    
    input1_label0 = input1_label0.append(df_pre_0)
    input2_label0 = input2_label0.append(df_post_0)  

In [ ]:
# 统一格式

input1_label0 = input1_label0.reset_index().set_index(['id','t'])
input2_label0 = input2_label0.reset_index().set_index(['id','t'])

input1_label1 = df_pre3.reset_index().set_index(['id','t'])
input2_label1 = df_post3.reset_index().set_index(['id','t'])

## 定义函数：生成输入张量、训练集和测试集

In [ ]:
def Tensorize(df) :  # 输入需要张量化的dataframe, df.index=['id','t'] df.values = ['features']
    # 样本个数
    num_sample = df.index.unique(0).shape[0]
    # 时间步长
    timesteps = df.index.unique(1).shape[0]
    # 特征数量
    num_features = df.shape[1]

    #张量化
    ar = df.to_numpy().reshape((num_sample, timesteps, num_features)) #(sample, timesteps, features)
    
    print('shape of array is : ', ar.shape)
    
    return ar

def split(df,test_size=0.2):
    
    ar = Tensorize(df)
    num = ar.shape[0]
    test_num = int(np.floor(num * test_size))
    
    ar_train = ar[test_num:]
    ar_test = ar[:test_num]
    
    return ar_train,ar_test


def get_data_for_model(input1_label1,input2_label1,input1_label0,input2_label0,features):

    #从label1数据中抽样

    # split training and test dataset
    # 不能随机打乱，因为input1 和 input2 有严格的对应关系    

    input1_l1_train, input1_l1_test = split(input1_label1[features])
    input2_l1_train, input2_l1_test = split(input2_label1[features])

    input1_l0_train, input1_l0_test = split(input1_label0[features])
    input2_l0_train, input2_l0_test = split(input2_label0[features])

    input1_train = np.concatenate( (input1_l1_train, input1_l0_train), axis = 0)
    input2_train = np.concatenate( (input2_l1_train, input2_l0_train), axis = 0)
    label_train = np.concatenate( (np.ones(input1_l1_train.shape[0]), np.zeros(input1_l0_train.shape[0]) ), axis = 0)

    input1_test = np.concatenate(  (input1_l1_test, input1_l0_test), axis = 0)
    input2_test = np.concatenate(  (input2_l1_test, input2_l0_test), axis = 0)
    label_test = np.concatenate( (np.ones(input1_l1_test.shape[0]), np.zeros(input1_l0_test.shape[0]) ), axis = 0)

    return input1_train,input2_train,label_train,input1_test,input2_test,label_test

## 搭建模型（以两层Bi-LSTM为例）

In [ ]:

def train_model(input1_train, input2_train, label_train, num_features, epochs=6):

    input1 = Input(shape=(None,num_features), dtype='float64', name='input1') # timesteps 长度可变
    lstm11 = Bidirectional(LSTM(128, return_sequences=True))(input1)
    lstm12 = Bidirectional(LSTM(128))(lstm11)

    input2 = Input(shape=(None,num_features), dtype='float64', name='input2')
    lstm21 = Bidirectional(LSTM(128, return_sequences=True))(input2)
    lstm22 = Bidirectional(LSTM(128))(lstm21)


    x = layers.concatenate([lstm12, lstm22])

    # 堆叠多个全连接网络层
    x = Dense(64, activation='relu')(x)
    x = Dense(32, activation='relu')(x)

    # 最后添加主要的逻辑回归层
    output = Dense(1, activation='sigmoid', name='output')(x)

    model = Model([input1, input2], output)
    #model.summary()
    
    #model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=[AUC(name='auc')])
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    history = model.fit([input1_train, input2_train], label_train, epochs=epochs, batch_size=64)
    
    return model,history


def get_training_result(history):
# 绘制结果

    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(acc) + 1)

    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))

    plt.subplot(1,2,1)
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

def get_test_result(model,input1_test,input2_test,label_test):
    
    # evaluate on test dataset
    print("Evaluate on test data")
    results = model.evaluate([input1_test,input2_test],label_test, batch_size=64)
    print("test loss, test acc:", results)

## 训练模型：以输入（x,y,size)为例

In [ ]:
input1_train_c,input2_train_c,label_train_c,input1_test_c,input2_test_c,label_test_c = get_data_for_model(
    input1_label1,input2_label1,input1_label0,input2_label0,
    features = ['x','y','size'])

model_c,history_c = train_model(input1_train_c, input2_train_c, label_train_c, num_features = 3,epochs = 6)
get_test_result(model_c,input1_test_c, input2_test_c,label_test_c)

# 在真实碰撞情况中测试

In [ ]:
# 构造需要判断的轨迹input

li = []

for i in range(len(id_link_filter)):
    

    #获取id
    id1 = id_link_filter.loc[i,'id1']
    id2 = id_link_filter.loc[i,'id2']
    id5 = id_link_filter.loc[i,'id5']
    id6 = id_link_filter.loc[i,'id6']
    
    df = data_clean
    

    #获取轨迹
    tra1 = df[df['id'] == id1].set_index('id').sort_values('t')
    tra2 = df[df['id'] == id2].set_index('id').sort_values('t')

    tra5 = df[df['id'] == id5].set_index('id').sort_values('t')
    tra6 = df[df['id'] == id6].set_index('id').sort_values('t')

    #构造input
    t1 = pd.concat([tra1.sub(tra1.tail(1))[['t','x','y']],tra1[['size','ovality']]],axis = 1).tail(30)
    t2 = pd.concat([tra2.sub(tra2.tail(1))[['t','x','y']],tra2[['size','ovality']]],axis = 1).tail(30)
    t5 = pd.concat([tra5.sub(tra5.head(1))[['t','x','y']],tra5[['size','ovality']]],axis = 1).head(30).tail(29)
    t6 = pd.concat([tra6.sub(tra6.head(1))[['t','x','y']],tra6[['size','ovality']]],axis = 1).head(30).tail(29)

    

    input1 = pd.concat([t1,t5])
    input2 = pd.concat([t1,t6])
    input3 = pd.concat([t2,t5])
    input4 = pd.concat([t2,t6])
    
    li.append(input1)
    li.append(input2)
    li.append(input3)
    li.append(input4)

#整合data
#x_concat = pd.concat([input1,input2,input3,input4],keys = range(4),names=['sample'])

x_concat = pd.concat(li,keys = range(len(li)),names=['sample'])  
    
x_test = x_concat.reset_index().set_index (['sample','t'])[['x','y','size','ovality']].sort_index()

In [ ]:
# 输入模型
x_test2 = Tensorize(x_test[['x','y','size']])
predict = pd.DataFrame(model.predict(x_test2))

In [ ]:
#按重叠事件整理
# 模型概率值
final_result = pd.concat([predict.iloc[0::4].reset_index(drop = True),predict.iloc[1::4].reset_index(drop = True),
               predict.iloc[2::4].reset_index(drop = True),predict.iloc[3::4].reset_index(drop = True)],
               axis = 1)
final_result.columns = ['1','3','4','2'] # ['15','16','25','26']
final_result = final_result[['1','2','3','4']]
final_result.columns = ['tra1','tra2','tra3','tra4']

# 0，1分类 thredhold = 0.5

predict_01 = predict.copy()
predict_01.columns = ['value']
predict_01.loc[predict_01['value'] > 0.5] = 1
predict_01.loc[predict_01['value'] <= 0.5] = 0

final_result_01 = pd.concat([predict_01.iloc[0::4].reset_index(drop = True),predict_01.iloc[1::4].reset_index(drop = True),
               predict_01.iloc[2::4].reset_index(drop = True),predict_01.iloc[3::4].reset_index(drop = True)],
               axis = 1)
final_result_01.columns = ['1','3','4','2']
final_result_01 = final_result_01[['1','2','3','4']]
final_result_01.columns = ['tra1','tra2','tra3','tra4']

In [ ]:
final_result_01.head(10)

In [ ]:
final_result.head(10).round(2)